In [1]:
import os
import sys
from functools import singledispatch
import warnings

import pandas as pd
import numpy as np

sys.path.insert(0, "/home/tthatcher/Desktop/Projects/Plio/plio")

from plio.examples import get_path
from plio.io.io_bae import read_gpf, read_ipf

/home/tthatcher/anaconda3/envs/autocnet/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Reads a .atf file and outputs all of the 
# .ipf, .gpf, .sup, .prj, and path to locate the 
# .apf file (should be the same as all others) 
def read_atf(atf_file):
    with open(atf_file) as f:

        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)
        
        # Creates appropriate arrays for certain files in the right format
        for file in files:
            file = file.strip()
            file = file.split(' ')

            # Grabs all the IPF files
            if file[1].endswith('.ipf'):
                ipf.append(file[1])

            # Grabs all the SUP files
            if file[1].endswith('.sup'):
                sup.append(file[1])

            files_dict.append(file)

        # Creates a dict out of file lists for GPF, PRJ, IPF, and ATF
        files_dict = (dict(files_dict))
        
        # Sets the value of IMAGE_IPF to all IPF images
        files_dict['IMAGE_IPF'] = ipf
        
        # Sets the value of IMAGE_SUP to all SUP images
        files_dict['IMAGE_SUP'] = sup
        
        # Sets the value of PATH to the path of the ATF file
        files_dict['PATH'] = os.path.dirname(os.path.abspath(atf_file))
        
        return files_dict

In [3]:
atf_dict = read_atf(get_path('CTX_Athabasca_Middle_step0.atf'))
print(atf_dict)

gpf_file = os.path.join(atf_dict['PATH'], atf_dict['GP_FILE']);
ipf_list = [os.path.join(atf_dict['PATH'], i) for i in atf_dict['IMAGE_IPF']]

gpf_df = read_gpf(gpf_file).set_index('point_id')
ipf_df = read_ipf(ipf_list).set_index('pt_id')

point_diff = ipf_df.index.difference(gpf_df.index)

if len(point_diff) != 0:
    warnings.warn("The following points found in ipf files missing from gpf file: \n\n{}. \
                  \n\nContinuing, but these points will be missing from the control network".format(list(point_diff)))

new_df = ipf_df.merge(gpf_df, left_index=True, right_index=True)
list(new_df)

{'PROJECT': 'D:\\data\\CTX_Athabasca_Middle.prj', 'ATF_FILE': 'CTX_Athabasca_Middle_step0.atf', 'GP_FILE': 'CTX_Athabasca_Middle.gpf', 'IMAGE_SUP': ['P19_008344_1894_XN_09N203W.sup', 'P20_008845_1894_XN_09N203W.sup', 'P03_002371_1888_XI_08N204W.sup', 'P01_001540_1889_XI_08N204W.sup', 'P01_001606_1897_XI_09N203W.sup', 'P03_002226_1895_XI_09N203W.sup'], 'IMAGE_IPF': ['P19_008344_1894_XN_09N203W.ipf', 'P20_008845_1894_XN_09N203W.ipf', 'P03_002371_1888_XI_08N204W.ipf', 'P01_001540_1889_XI_08N204W.ipf', 'P01_001606_1897_XI_09N203W.ipf', 'P03_002226_1895_XI_09N203W.ipf'], 'PATH': '/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet'}


/home/tthatcher/anaconda3/envs/autocnet/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The following points found in ipf files missing from gpf file: 

['P03_002226_1895_XI_09N203W_15', 'P03_002226_1895_XI_09N203W_16', 'P03_002226_1895_XI_09N203W_17', 'P03_002226_1895_XI_09N203W_18', 'P03_002226_1895_XI_09N203W_19', 'P03_002226_1895_XI_09N203W_20', 'P03_002226_1895_XI_09N203W_21', 'P03_002226_1895_XI_09N203W_22', 'P03_002226_1895_XI_09N203W_24', 'P03_002226_1895_XI_09N203W_26', 'P03_002226_1895_XI_09N203W_30', 'P03_002226_1895_XI_09N203W_31', 'P03_002226_1895_XI_09N203W_32', 'P03_002226_1895_XI_09N203W_34', 'P03_002226_1895_XI_09N203W_36', 'P03_002226_1895_XI_09N203W_37', 'P03_002226_1895_XI_09N203W_44', 'P03_002226_1895_XI_09N203W_48', 'P03_002226_1895_XI_09N203W_49', 'P03_002226_1895_XI_09N203W_56', 'P03_002226_1895_XI_09N203W_57', 'P03_002226_1895_XI_09N203W_61', 'P03_002226_1895_XI_09N203W_62', 'P03_002226_1895_XI_09N203W_63', 'P03_002226_1895_XI_09N203W_65', 'P

['val',
 'fid_val',
 'no_obs',
 'l.',
 's.',
 'sig_l',
 'sig_s',
 'res_l',
 'res_s',
 'fid_x',
 'fid_y',
 'ipf_file',
 'stat',
 'known',
 'lat_Y_North',
 'long_X_East',
 'ht',
 'sig0',
 'sig1',
 'sig2',
 'res0',
 'res1',
 'res2']

In [4]:
def sample_size(record):
    
    with open(atf_dict['PATH'] + '/' + record['ipf_file'] + '.sup') as f:
        sample_num = 0
        for i, line in enumerate(f):
            if i == 4:
                sample_num = line.split(' ')
                sample_num = sample_num[-1].strip()
                break
        assert int(sample_num) > 0, "Sample number {} from {} is a negative number: Invalid Data".format(sample_num, record['ipf_file'])
            
        sample_num = int(sample_num)/2.0 + record['s.'] + 1
        return sample_num

def line_size(record):
    
    with open(atf_dict['PATH'] + '/' + record['ipf_file'] + '.sup') as f:
        sample_num = 0
        for i, line in enumerate(f):
            if i == 3:
                sample_num = line.split(' ')
                sample_num = sample_num[-1].strip()
                break
        assert int(sample_num) > 0, "Sample number {} from {} is a negative number: Invalid Data".format(sample_num, record['ipf_file'])
            
        sample_num = int(sample_num)/2.0 + record['l.'] + 1
        return sample_num

def known(record):
    if record['known'] == 0:
        return 'Free'
    
    elif record['known'] == 1 or record['known'] == 2 or record['known'] == 3:
        return 'Constrained'
    
# new_df['s.'] = new_df.apply(sample_size, axis=1)
# new_df['l.'] = new_df.apply(line_size, axis=1)
# new_df['known'] = new_df.apply(known, axis=1)
display(new_df)
# cmeas->SetCoordinate(img_samples/2.0+sample+1,img_lines/2.0+line+1);

,val,fid_val,no_obs,l.,s.,sig_l,sig_s,res_l,res_s,fid_x,...,known,lat_Y_North,long_X_East,ht,sig0,sig1,sig2,res0,res1,res2
10_8344_8845_4r,1,0,0,-2919.380615,1052.729004,0.000000,0.000000,0.059914,0.930311,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,-4044.697510,1008.950928,0.000000,0.000000,0.063678,0.661294,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,1700.584473,-2390.001709,0.000000,0.000000,-0.063695,-1.055619,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,2006.141113,-2234.915283,0.000000,0.000000,0.318779,-0.585138,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
11_8344_8845_4r,1,0,0,370.151917,2340.653076,0.000000,0.000000,-0.239226,-1.113693,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,-761.216064,2303.787109,0.000000,0.000000,-0.247307,-0.587299,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,4985.437988,-1070.364990,0.000000,0.000000,0.239207,1.080801,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,5293.700195,-993.390625,0.000000,0.000000,0.672877,0.626316,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
12_8344_8845_4r,1,0,0,236.395218,984.833252,0.000000,0.000000,0.073683,-0.237114,0.0,...,0,0.164949,2.724076,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858
12_8344_8845_4r,1,0,0,-889.364441,966.533997,0.000000,0.000000,-0.251753,0.008170,0.0,...,0,0.164949,2.724076,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858


In [7]:
import math
import pyproj

# converts +/- 180 system to 0 - 360 system
def to_360(num):
    return num % 360

# ocentric to ographic latitudes
def oc2og(dlat, dMajorRadius, dMinorRadius):
    try:    
        dlat = math.radians(dlat)
        dlat = math.atan(((dMajorRadius / dMinorRadius)**2) * (math.tan(dlat)))
        dlat = math.degrees(dlat)
    except:
        print ("Error in oc2og conversion")
    return dlat

# ographic to ocentric latitudes
def og2oc(dlat, dMajorRadius, dMinorRadius):
    try:
        dlat = math.radians(dlat)
        dlat = math.atan((math.tan(dlat) / ((dMajorRadius / dMinorRadius)**2)))
        dlat = math.degrees(dlat)
    except:
        print ("Error in og2oc conversion")
    return dlat

def get_axis():
    with open(atf_dict['PATH'] + '/' + 'CTX_Athabasca_Middle.prj') as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        
        return eRadius, pRadius
    
# function to convert lat_Y_North to ISIS_lat
def lat_ISIS_coord(record):
    with open(atf_dict['PATH'] + '/' + 'CTX_Athabasca_Middle.prj') as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        ocentric_coord = og2oc(record['lat_Y_North'], eRadius, pRadius)
        coord_360 = to_360(ocentric_coord)
        return coord_360

# function to convert long_X_East to ISIS_lon
def lon_ISIS_coord(record):
    with open(atf_dict['PATH'] + '/' + 'CTX_Athabasca_Middle.prj') as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        ocentric_coord = og2oc(record['long_X_East'], eRadius, pRadius)
        coord_360 = to_360(ocentric_coord)
        return coord_360
            
def body_fixed_lat(record):
    semi_major, semi_minor = get_axis() 
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return lat

def body_fixed_lon(record):
    semi_major, semi_minor = get_axis() 
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return lon

def body_fixed_height(record):
    semi_major, semi_minor = get_axis() 
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return height


new_df['lat_Y_North'] = new_df.apply(lat_ISIS_coord, axis=1)
new_df['long_X_East'] = new_df.apply(lon_ISIS_coord, axis=1)

new_df['lat_Y_North'] = new_df.apply(body_fixed_lat, axis=1)
new_df['long_X_East'] = new_df.apply(body_fixed_lon, axis=1)

new_df['ht'] = new_df.apply(body_fixed_height, axis=1)


display(new_df)

Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2

Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2og conversion
Error in oc2

inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf


,val,fid_val,no_obs,l.,s.,sig_l,sig_s,res_l,res_s,fid_x,...,known,lat_Y_North,long_X_East,ht,sig0,sig1,sig2,res0,res1,res2
10_8344_8845_4r,1,0,0,-2919.380615,1052.729004,0.000000,0.000000,0.059914,0.930311,0.0,...,0,inf,inf,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,-4044.697510,1008.950928,0.000000,0.000000,0.063678,0.661294,0.0,...,0,inf,inf,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,1700.584473,-2390.001709,0.000000,0.000000,-0.063695,-1.055619,0.0,...,0,inf,inf,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,2006.141113,-2234.915283,0.000000,0.000000,0.318779,-0.585138,0.0,...,0,inf,inf,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
11_8344_8845_4r,1,0,0,370.151917,2340.653076,0.000000,0.000000,-0.239226,-1.113693,0.0,...,0,inf,inf,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,-761.216064,2303.787109,0.000000,0.000000,-0.247307,-0.587299,0.0,...,0,inf,inf,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,4985.437988,-1070.364990,0.000000,0.000000,0.239207,1.080801,0.0,...,0,inf,inf,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,5293.700195,-993.390625,0.000000,0.000000,0.672877,0.626316,0.0,...,0,inf,inf,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
12_8344_8845_4r,1,0,0,236.395218,984.833252,0.000000,0.000000,0.073683,-0.237114,0.0,...,0,inf,inf,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858
12_8344_8845_4r,1,0,0,-889.364441,966.533997,0.000000,0.000000,-0.251753,0.008170,0.0,...,0,inf,inf,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858


In [ ]:
# Reads a .atf file and outputs all of the 
# .ipf, .gpf, .sup, .prj, and path to locate the 
# .apf file (should be the same as all others) 
def read_atf(atf_file):
    with open(atf_file) as f:

        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)
        
        # Creates appropriate arrays for certain files in the right format
        for file in files:
            file = file.strip()
            file = file.split(' ')

            # Grabs all the IPF files
            if file[1].endswith('.ipf'):
                ipf.append(file[1])

            # Grabs all the SUP files
            if file[1].endswith('.sup'):
                sup.append(file[1])

            files_dict.append(file)

        # Creates a dict out of file lists for GPF, PRJ, IPF, and ATF
        files_dict = (dict(files_dict))
        
        # Sets the value of IMAGE_IPF to all IPF images
        files_dict['IMAGE_IPF'] = ipf
        
        # Sets the value of IMAGE_SUP to all SUP images
        files_dict['IMAGE_SUP'] = sup
        
        # Sets the value of PATH to the path of the ATF file
        files_dict['PATH'] = os.path.dirname(os.path.abspath(atf_file))
        
        return files_dict
    
@singledispatch
def read_ipf(arg):
    return str(arg)
# new_df['known'] = new_df.apply(known, axis=1)

@read_ipf.register(str)
def read_ipf_str(input_data):
    """
    Read a socet ipf file into a pandas data frame

    Parameters
    ----------
    input_data : str
                 path to the an input data file

    Returns
    -------
    df : pd.DataFrame
         containing the ipf data with appropriate column names and indices
    """

    # Check that the number of rows is matching the expected number
    with open(input_data, 'r') as f:
        for i, l in enumerate(f):
            if i == 1:
                cnt = int(l)
            elif i == 2:
                col = l
                break
                
    columns = np.genfromtxt(input_data, skip_header=2, dtype='unicode',
                            max_rows = 1, delimiter = ',')

    # TODO: Add unicode conversion
    d = [line.split() for line in open(input_data, 'r')]
    d = np.hstack(np.array(d[3:]))
    
    d = d.reshape(-1, 12)
    
    df = pd.DataFrame(d, columns=columns)
    file = os.path.split(os.path.splitext(input_data)[0])[1]
    df['ipf_file'] = pd.Series(np.full((len(df['pt_id'])), file), index = df.index)

    assert int(cnt) == len(df), 'Dataframe length {} does not match point length {}.'.format(int(cnt), len(df))
    
    # Soft conversion of numeric types to numerics, allows str in first col for point_id
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

@read_ipf.register(list)
def read_ipf_list(input_data_list):
    """
    Read a socet ipf file into a pandas data frame

    Parameters
    ----------
    input_data_list : list
                      list of paths to the a set of input data files

    Returns
    -------
    df : pd.DataFrame
         containing the ipf data with appropriate column names and indices
    """
    frames = []

    for input_file in input_data_list:
        frames.append(read_ipf(input_file))

    df = pd.concat(frames)

    return df